In [ ]:
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers

In [ ]:
print(tf. __version__)

2.12.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255.,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   fill_mode="nearest")

In [ ]:
# Training folder location
train_dir = '/content/drive/MyDrive/C23-PC609 | Product-Based/Machine_Learning/Image_dataset_binary/Train'

# Using TrainGenerator to label the data and set target size
train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    # color_mode='grayscale',
    batch_size=16,
    class_mode='categorical'
)

Found 1442 images belonging to 2 classes.


In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255.)

In [ ]:
# Validation folder location
validation_dir = '/content/drive/MyDrive/C23-PC609 | Product-Based/Machine_Learning/Image_dataset_binary/Validation'

# Using flow_from_directory to label the data and set target size
val_gen = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    # color_mode='grayscale',
    batch_size=16,
    class_mode='categorical'
)

Found 145 images belonging to 2 classes.


### Build the Model (Model 3)
- CNN layer (Convolutional and MaxPooling)
- Using Regularization l2
- Using 2 dropout layer (0.5)
- Compile using adam (lr=0.001) optimizer

In [ ]:
# Try Model 3 : Add dropout layer and value (0.5) + declare learning rate = 0.001
def createModel3():
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=(150, 150, 3)),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dropout(0.5),
      tf.keras.layers.Dense(512, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.02)),
      tf.keras.layers.Dropout(0.5),
      tf.keras.layers.Dense(2, activation="softmax")
  ])

  # Compile the Model
  optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
  model.compile(optimizer=optimizer,
                loss="categorical_crossentropy",
                metrics=["accuracy"])

  return model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Create the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

# Custom condition for stopping based on validation accuracy
class CustomEarlyStopping(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs['val_accuracy'] >= 0.85:
            self.model.stop_training = True

# Create an instance of the custom early stopping callback
custom_early_stopping = CustomEarlyStopping()
model3 = createModel3()

# Train the model with custom early stopping
history3 = model3.fit(train_gen,
                    epochs=100,
                    validation_data=val_gen,
                    callbacks=[early_stopping, custom_early_stopping])

Epoch 1/100
91/91 [==============================] - 397s 4s/step - loss: 1.9271 - accuracy: 0.8426 - val_loss: 1.1886 - val_accuracy: 0.7655
Epoch 2/100
91/91 [==============================] - 125s 1s/step - loss: 0.5067 - accuracy: 0.8904 - val_loss: 1.4002 - val_accuracy: 0.7793
Epoch 3/100
91/91 [==============================] - 123s 1s/step - loss: 0.4750 - accuracy: 0.8883 - val_loss: 1.4178 - val_accuracy: 0.7793
Epoch 4/100
91/91 [==============================] - 114s 1s/step - loss: 0.4278 - accuracy: 0.9022 - val_loss: 1.0430 - val_accuracy: 0.7862
Epoch 5/100
91/91 [==============================] - 117s 1s/step - loss: 0.4085 - accuracy: 0.9119 - val_loss: 1.3038 - val_accuracy: 0.7724
Epoch 6/100
91/91 [==============================] - 116s 1s/step - loss: 0.3901 - accuracy: 0.9057 - val_loss: 1.7485 - val_accuracy: 0.6690
Epoch 7/100
91/91 [==============================] - 117s 1s/step - loss: 0.3882 - accuracy: 0.9050 - val_loss: 0.8423 - val_accuracy: 0.8069
Epoch 

In [ ]:
# Save the model
model3.save('/content/drive/MyDrive/C23-PC609 | Product-Based/Machine_Learning/model3_binary_categorical.h5')

### Convert to tflite
Convert h5 file to tflite (reduce the size & using it on android app) using tfliteconverter

In [ ]:
# Load the .h5 model
model = tf.keras.models.load_model('/content/drive/MyDrive/C23-PC609 | Product-Based/Machine_Learning/model3_binary_categorical.h5')

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
with open('/content/drive/MyDrive/C23-PC609 | Product-Based/Machine_Learning/model3_binary_categorical.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
# Load Model
from tensorflow.keras.models import load_model

model = load_model("/content/drive/MyDrive/C23-PC609 | Product-Based/Machine_Learning/model3_binary_categorical.h5")

In [ ]:
from google.colab import files
from tensorflow import keras
from keras.utils import load_img, img_to_array

uploaded = files.upload()

for fn in uploaded.keys():
    # predicting images
    path = fn
    img = load_img(path, target_size=(150, 150))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x / 255.0

    images = np.vstack([x])
    classes = model.predict(images)
    # probability_rotten = classes[:10]
    # probability_fresh = classes[10:]
    # print(probability_rotten)
    # print(probability_fresh)
    idx_classes = np.argmax(classes)
    print("File name:",fn)
    print("Prediction result:",idx_classes)

Saving 005.jpg to 005 (1).jpg
1/1 [==============================] - 0s 18ms/step
File name: 005.jpg
Prediction result: 0
